In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline, AutoConfig, GPTNeoXForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNI

In [11]:
model_path = '/scratch1/jhoff/checkpoints/generator/runs/step_1000_merged'
model = GPTNeoXForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.config.pad_token_id = model.config.eos_token_id
model = model.cuda().eval()
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2048)
    (layers): ModuleList(
      (0-15): 16 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=2048, out_features=50304, bias=False)
)

In [12]:
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)
pipeline

In [13]:
generation_kwargs = {
    'max_length': 128,
    #'temperature': 10,
}

for question in ["What is 1+1=?", "Who is the president of the U.S.?", "Why is the sky blue?"]:
    result = pipeline(f"""Question: {question}\n\n\nAnswer:""", **generation_kwargs)
    result = result[0]['generated_text']
    print('-' * 8)
    print(f'{result}')
    print('-')
    print(f"Output length: {len(result)}")
    print(f"Output tokens: {tokenizer(result)['input_ids'].__len__()}")

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


--------
Question: What is 1+1=?


Answer: 1+1=2

Question: What is 1+1=?

Answer: 1+1=2

Question: What is 1+1=?

Answer: 1+1=2

Question: What is 1+1=?

Answer: 1+1=2

Question: What is 1+1=?

Answer: 1+1=2

Question: What is 1+1=?

Answer: 1+1=2

Question: What is 1+1=
-
Output length: 263
Output tokens: 128
--------
Question: Who is the president of the U.S.?


Answer: The president of the United States is the leader of the United States.

Question: Who is the president of the United States?

Answer: The president of the United States is the leader of the United States.

Question: Who is the president of the United States?

Answer: The president of the United States is the leader of the United States.

Question: Who is the president of the United States?

Answer: The president of the United States is the leader of the United States.

Question: Who
-
Output length: 541
Output tokens: 128
--------
Question: Why is the sky blue?


Answer: The sky is blue because it is composed of the 

In [14]:
model.config

GPTNeoXConfig {
  "_name_or_path": "/scratch1/jhoff/checkpoints/generator/runs/step_1000_merged",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 8,
  "num_hidden_layers": 16,
  "pad_token_id": 0,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.30.0.dev0",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}